# CSC495 - DataMining - HW#3 - Rinty Chowdhury

In [1]:
import pandas as pd
import csv
import gensim
from gensim.parsing.preprocessing import preprocess_string
import gensim.corpora as corpora
from gensim.parsing.preprocessing import strip_tags, strip_punctuation, strip_numeric, remove_stopwords, strip_multiple_whitespaces, strip_short
from nltk.stem import WordNetLemmatizer
from gensim.corpora import Dictionary
from gensim.models import LdaModel
from pprint import pprint

In [2]:
csv.field_size_limit(1000000000)
my_data = pd.read_csv('./state-of-the-union.csv', names=['Year', 'Text'])
my_data = my_data.replace('\n',' ', regex=True)
my_data = my_data.groupby(['Year'])['Text'].apply(', '.join).reset_index()

In [5]:
century_20th = my_data[(my_data.Year >= 1901) & (my_data.Year <= 2000)].reset_index()
del century_20th['index']

century_21st = my_data[(my_data.Year >= 2001) & (my_data.Year <= 2012)].reset_index()
del century_21st['index']

In [6]:
decade1_20 = century_20th[(century_20th.Year >= 1901) & (century_20th.Year <= 1910)]
del decade1_20['Year']
decade1_20 = decade1_20['Text'].str.cat(sep=', ')

decade2_20 = century_20th[(century_20th.Year >= 1911) & (century_20th.Year <= 1920)]
del decade2_20['Year']
decade2_20 = decade2_20['Text'].str.cat(sep=', ')

decade3_20 = century_20th[(century_20th.Year >= 1921) & (century_20th.Year <= 1930)]
del decade3_20['Year']
decade3_20 = decade3_20['Text'].str.cat(sep=', ')

decade4_20 = century_20th[(century_20th.Year >= 1931) & (century_20th.Year <= 1940)]
del decade4_20['Year']
decade4_20 = decade4_20['Text'].str.cat(sep=', ')

decade5_20 = century_20th[(century_20th.Year >= 1941) & (century_20th.Year <= 1950)]
del decade5_20['Year']
decade5_20 = decade5_20['Text'].str.cat(sep=', ')

decade6_20 = century_20th[(century_20th.Year >= 1951) & (century_20th.Year <= 1960)]
del decade6_20['Year']
decade6_20 = decade6_20['Text'].str.cat(sep=', ')

decade7_20 = century_20th[(century_20th.Year >= 1961) & (century_20th.Year <= 1970)]
del decade7_20['Year']
decade7_20 = decade7_20['Text'].str.cat(sep=', ')

decade8_20 = century_20th[(century_20th.Year >= 1971) & (century_20th.Year <= 1980)]
del decade8_20['Year']
decade8_20 = decade8_20['Text'].str.cat(sep=', ')

decade9_20 = century_20th[(century_20th.Year >= 1981) & (century_20th.Year <= 1990)]
del decade9_20['Year']
decade9_20 = decade9_20['Text'].str.cat(sep=', ')

decade10_20 = century_20th[(century_20th.Year >= 1991) & (century_20th.Year <= 2000)]
del decade10_20['Year']
decade10_20 = decade10_20['Text'].str.cat(sep=', ')

d1 = {'Decade': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'Text': [decade1_20, decade2_20, 
                decade3_20, decade4_20, decade5_20, decade6_20, decade7_20, decade8_20, decade9_20, decade10_20]}
decade_20th_century = pd.DataFrame(data=d1)

In [7]:
decade1_21 = century_21st[(century_21st.Year >= 2001) & (century_21st.Year <= 2010)]
del decade1_21['Year']
decade1_21 = decade1_21['Text'].str.cat(sep=', ')

decade2_21 = century_21st[(century_21st.Year >= 2011) & (century_21st.Year <= 2012)]
del decade2_21['Year']
decade2_21 = decade2_21['Text'].str.cat(sep=', ')

d2 = {'Decade': [1, 2], 'Text': [decade1_21, decade2_21]}
decade_21st_century = pd.DataFrame(data=d2)

In [ ]:
for i in range (0, len(my_data['Text'])):
    my_data.Text[i] = str(my_data.Text[i])

for i in range (0, len(decade_20th_century['Text'])):
    decade_20th_century.Text[i] = str(decade_20th_century.Text[i])

for i in range (0, len(decade_21st_century['Text'])):
    decade_21st_century.Text[i] = str(decade_21st_century.Text[i])

In [ ]:
CUSTOM_FILTERS = [lambda x: x.lower(), strip_tags, strip_punctuation, strip_numeric, remove_stopwords, 
                  strip_multiple_whitespaces, strip_short]

for i in range (0, len(my_data['Text'])):
    my_data.Text[i] = preprocess_string(my_data.Text[i], CUSTOM_FILTERS)
    
for i in range (0, len(decade_20th_century['Text'])):
    decade_20th_century.Text[i] = preprocess_string(decade_20th_century.Text[i], CUSTOM_FILTERS)
    
for i in range (0, len(decade_21st_century['Text'])):
    decade_21st_century.Text[i] = preprocess_string(decade_21st_century.Text[i], CUSTOM_FILTERS)

In [10]:
lemmatizer = WordNetLemmatizer()

for m in range (0, len(my_data['Text'])):
    for n in range (0, len(my_data.Text[m])):
        my_data.Text[m][n] = lemmatizer.lemmatize(my_data.Text[m][n])
        
for m in range (0, len(decade_20th_century['Text'])):
    for n in range (0, len(decade_20th_century.Text[m])):
        decade_20th_century.Text[m][n] = lemmatizer.lemmatize(decade_20th_century.Text[m][n])
        
for m in range (0, len(decade_21st_century['Text'])):
    for n in range (0, len(decade_21st_century.Text[m])):
        decade_21st_century.Text[m][n] = lemmatizer.lemmatize(decade_21st_century.Text[m][n])

In [11]:
data = my_data['Text']
data_20th = decade_20th_century['Text']
data_21st = decade_21st_century['Text']

In [12]:
id2word = Dictionary(data)
id2word.filter_extremes(no_below=20, no_above=0.5)
corpus = [id2word.doc2bow(text) for text in data]

id2word_20th = Dictionary(data_20th)
id2word_20th.filter_extremes(no_below=4, no_above=0.5)
corpus_20th = [id2word_20th.doc2bow(text) for text in data_20th]

id2word_21st = Dictionary(data_21st)
corpus_21st = [id2word_21st.doc2bow(text) for text in data_21st]

In [13]:
lda_model = LdaModel(corpus=corpus, id2word=id2word, num_topics=20, iterations = 400, chunksize=2000,
                    passes=10, alpha='auto', eta='auto', per_word_topics=True, eval_every = None)

lda_model_20th = LdaModel(corpus=corpus_20th, id2word=id2word_20th, num_topics=10, iterations = 400, chunksize=2000,
                    passes=10, alpha='auto', eta='auto', per_word_topics=True, eval_every = None)

lda_model_21st = LdaModel(corpus=corpus_21st, id2word=id2word_21st, num_topics=2, iterations = 400, chunksize=2000,
                    passes=10, alpha='auto', eta='auto', per_word_topics=True, eval_every = None)

In [14]:
pprint(lda_model.print_topics())

[(0,
  '0.006*"program" + 0.005*"resolution" + 0.005*"requires" + 0.005*"today" + '
  '0.005*"productive" + 0.005*"area" + 0.004*"attitude" + 0.004*"humanity" + '
  '0.004*"neighbor" + 0.004*"capability"'),
 (1,
  '0.025*"program" + 0.011*"billion" + 0.007*"budget" + 0.007*"farm" + '
  '0.006*"area" + 0.006*"level" + 0.005*"wage" + 0.005*"today" + 0.005*"major" '
  '+ 0.005*"help"'),
 (2,
  '0.014*"help" + 0.013*"job" + 0.012*"tonight" + 0.009*"school" + '
  '0.009*"budget" + 0.007*"cut" + 0.007*"program" + 0.006*"today" + '
  '0.006*"challenge" + 0.005*"deficit"'),
 (3,
  '0.015*"enemy" + 0.013*"fighting" + 0.009*"attack" + 0.009*"victory" + '
  '0.008*"japanese" + 0.006*"air" + 0.006*"job" + 0.006*"german" + '
  '0.006*"democracy" + 0.006*"battle"'),
 (4,
  '0.009*"gold" + 0.007*"note" + 0.007*"currency" + 0.005*"silver" + '
  '0.004*"circulation" + 0.004*"bond" + 0.003*"tariff" + 0.003*"mexico" + '
  '0.003*"paper" + 0.003*"article"'),
 (5,
  '0.004*"intercourse" + 0.004*"tribe" + 0

### Description of the top 20 topics:
0.  It’s talking about a vision that requires all American to help their neighbors and be productive in their area.
1.	This topic is related to allocating budget in the farm sector which will increase the wages of the workers.
2.	this topic is related to a cut in the budget for school related fund which poses a challenge to skillful workforce.
3.	It’s talking about Pearl Harbor attack when the Japanese attacked US battleship. They joined the German Nazi force to fight against US in WWII.
4.	This topic is related to the federal reserve of currency, coins (gold & silver) and bond. It’s also talking about tariffs to be imposed for trade with Mexico.
5.	Spanish Army invaded Mexican region and colonized mating with the tribes.
6.	Due to mismanagement allegation in housing, veterans collectively went to a strike.
7.	When export vs import imbalance increased, the country went to a depression. Wages also remained stagnant.
8.	Mexico was at war with Spain for independence. Texas and California became annexed to US.
9.	No logical conclusion can be drawn
10. With the rapid industrial growth, people with less machine skills are fearful of losing their jobs.
11. In this Capitalist economic boom, the industrial big corporations became wealthy while average worker’s wage merely increased.
12. During the nuclear arms race with Soviet Union, government assistance was provided to oil companies to provide fuel for US forces.
13. No logical conclusion can be drawn
14.	This is discussing three separate topics. One of them is relationship with Cuba. The other one is political convention. The last one is federal reserve of currency, coin, and bond.
15.	Tonight, the President is describing how billions allocated to the budget will achieve unprecedented level of job growth.
16.	Similar to topic 11.
17.	Federal bureau of investigation was investigating some cases of tax evasion by US companies in Panama.
18.	Both Soviet and Korean communist rulers made significant improvement of nation-wide railway and industrial complex. But farmers suffered under their regime.
19.	No logical conclusion can be drawn.

In [15]:
pprint(lda_model_20th.print_topics())

[(0,
  '0.001*"atomic" + 0.001*"missile" + 0.001*"wartime" + 0.001*"ruler" + '
  '0.001*"manpower" + 0.001*"answered" + 0.001*"satellite" + 0.001*"nuclear" + '
  '0.001*"aggressor" + 0.001*"ballistic"'),
 (1,
  '0.010*"speculation" + 0.008*"ruler" + 0.008*"livelihood" + '
  '0.008*"definitely" + 0.008*"franklin" + 0.008*"compelled" + '
  '0.006*"intelligent" + 0.006*"neutrality" + 0.006*"panic" + '
  '0.006*"sentiment"'),
 (2,
  '0.046*"tonight" + 0.028*"nuclear" + 0.012*"vietnam" + 0.011*"missile" + '
  '0.009*"sector" + 0.008*"medicare" + 0.007*"funding" + 0.007*"nato" + '
  '0.007*"capability" + 0.005*"classroom"'),
 (3,
  '0.015*"arbitration" + 0.009*"cruiser" + 0.006*"tribunal" + 0.006*"manifest" '
  '+ 0.006*"objection" + 0.005*"rico" + 0.005*"commissioner" + '
  '0.005*"postmaster" + 0.005*"workman" + 0.005*"ordinary"'),
 (4,
  '0.001*"supervision" + 0.001*"enlisted" + 0.001*"colombia" + '
  '0.001*"arbitration" + 0.001*"practicable" + 0.001*"ordinary" + '
  '0.001*"disturbance"

### Description of decade summarization algorithm: First, I have created separate data frame for each century. One for 20th and another one for 21st. From the 20th century data frame, I created new str class for each decade and combined all the text of the decade into one row. I repeated the same process for all decades in 20th and 21st century data frame. Then I took all the decade class of a century and created a data frame using those str class. I have done the same thing for both centuries. Once I have the data frame ready, then I implemented the preprocessing of the data and LDA model.

### Analysis of the topics: The emerging pattern from these topics is that US wanted to portray itself as a superpower to the rest of the world. Topics highlights few major events in history such as World War I and II, Cold war with Soviet Union, Cuban Missile Crisis, Vietnam War, Drug war in South America, Conflict in Iran.

In [16]:
pprint(lda_model_21st.print_topics())

[(0,
  '0.010*"american" + 0.009*"year" + 0.008*"job" + 0.007*"new" + '
  '0.007*"america" + 0.007*"people" + 0.005*"business" + 0.005*"tax" + '
  '0.004*"world" + 0.004*"country"'),
 (1,
  '0.011*"america" + 0.010*"american" + 0.008*"year" + 0.008*"people" + '
  '0.007*"nation" + 0.006*"country" + 0.006*"new" + 0.006*"world" + '
  '0.005*"tax" + 0.005*"congress"')]
